<a href="https://colab.research.google.com/github/Whitchurch/Tensorflow/blob/main/Team17_CNN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install tensorflow-GPU

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Install Tensorflow on Google Co-lab

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import csv 
from tensorflow import keras
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers import AveragePooling2D
from keras.layers import Flatten
from keras.layers import CuDNNLSTM # A superior LSTM that uses GPU more optimized for training.
from keras.layers import LSTM
from keras.models import Sequential

Steps to pre-process and load the data

In [3]:
os.chdir("drive/My Drive/Colab Notebooks")


In [4]:
ls

 Access_Local_directory.ipynb
'Copy of Untitled0.ipynb'
 ECGsignal.csv
 ECGsignal_n.csv
 ECGsignal_nwhite.csv
 Exercise_1_Cats_vs_Dogs_Question-FINAL.ipynb
 Exercise_2_Cats_vs_Dogs_using_augmentation_Question-FINAL.ipynb
 Exercise_3_Horses_vs_humans_using_Transfer_Learning_Question-FINAL.ipynb
 Exercise_4_Multi_class_classifier_Question-FINAL.ipynb
 FinalCNN_Beginner.ipynb
 Team17_CNN_LSTM.ipynb
 Untitled0.ipynb
 Untitled1.ipynb
 Untitled2.ipynb


In [5]:
#Read the clean signal
with open('ECGsignal.csv') as csv_file:
  csv_reader = csv.reader(csv_file)

  for ECGsignal in csv_reader:
    print(ECGsignal)
  print("Finished reading the file")

#Time to dissect the file read data's anatomy:
#Total number of items in the list:
len(ECGsignal)

#Convert the list to an array 
ECGsignal_arr = np.array(ECGsignal,dtype=float)

#Read the noisy signal
with open('ECGsignal_n.csv') as csv_file:
  csv_reader = csv.reader(csv_file)

  for ECGsignal_n in csv_reader:
    print(ECGsignal_n)
  print("Finished reading the file")

#Time to dissect the file read data's anatomy:
#Total number of items in the list:
len(ECGsignal_n)

#Convert the list to an array 
ECGsignal_n_arr = np.array(ECGsignal_n,dtype=float)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
tupleofdimensions = ECGsignal_arr.shape
num = tupleofdimensions[0]
Training_test_Split = 0.80  # Modify this line to control the Test-Train split

a=Training_test_Split*num;                                 
XTrain=ECGsignal_n_arr[0:int(a)];                  
YTrain=ECGsignal_arr[0:int(a)];
XTest=ECGsignal_n_arr[int(a):num];               
YTest=ECGsignal_arr[int(a):num];

train_samples=30*60*Training_test_Split/10;
test_samples =30*60*(1-Training_test_Split)/10;

print(int(train_samples))
print(int(test_samples))

144
35


Break up the Train and Test data into 4D vectors for the Tensorflow CNN: Format is [Batch_size,Height,width,depth]

In [7]:
XTrain_reshaped = np.reshape(XTrain,(int(train_samples),30000,1,1))
YTrain_reshaped = np.reshape(YTrain,(int(train_samples),30000))

XTest_reshaped = np.reshape(XTest,(int(test_samples),30000,1,1))
YTest_reshaped = np.reshape(YTest,(int(test_samples),30000))



Start defining the layers of the CNN - From the De-noising paper

In [ ]:
model = Sequential()

# Convolution layers are used to extract the most prominent features of the input data.
#Layer 1:
#Conv-layer -> BatchNorm -> RELU-> AvdPooling
model.add(Conv2D(36, (19,1), input_shape=(30000,1,1),activation='relu',padding='SAME',strides=(1,1),name='conv_1')) 
model.add(BatchNormalization(name='batchnorm_1'))
model.add(Dense(units=36,activation='relu',name='relu_1'))
model.add(AveragePooling2D(pool_size=(2,1),strides=(4,1),name='avgpool_1'))

#Layer 2:
#Conv-layer -> BatchNorm -> RELU-> AvdPooling
model.add(Conv2D(36,(19,1),strides=(1,1),padding='SAME',name='conv_2'))
model.add(BatchNormalization(name='batchnorm_2'))
model.add(Dense(units=36,activation='relu',name='relu_2'))
model.add(AveragePooling2D(pool_size=(2,1),strides=(4,1),name='avgpool_2'))

#Layer 3:
#Conv-layer -> BatchNorm -> RELU-> AvdPooling
model.add(Conv2D(36,(19,1),strides=(1,1),padding='SAME',name='conv_3'))
model.add(BatchNormalization(name='batchnorm_3'))
model.add(Dense(units=36,activation='relu',name='relu_3'))
model.add(AveragePooling2D(pool_size=(2,1),strides=(4,1),name='avgpool_3'))

#Layer 4:
#Conv-layer -> BatchNorm -> RELU-> AvdPooling
model.add(Conv2D(36,(19,1),strides=(1,1),padding='SAME',name='conv_4'))
model.add(BatchNormalization(name='batchnorm_4'))
model.add(Dense(units=36,activation='relu',name='relu_4'))
model.add(AveragePooling2D(pool_size=(2,1),strides=(4,1),name='avgpool_4'))


#Layer 5:
#Conv-layer -> BatchNorm -> RELU-> AvdPooling
model.add(Conv2D(36,(19,1),strides=(1,1),padding='SAME',name='conv_5'))
model.add(BatchNormalization(name='batchnorm_5'))
model.add(Dense(units=36,activation='relu',name='relu_5'))
model.add(AveragePooling2D(pool_size=(2,1),strides=(4,1),name='avgpool_5'))


#Layer 6:
#Conv-layer -> BatchNorm -> RELU-> AvdPooling
model.add(Conv2D(36,(19,1),strides=(1,1),padding='SAME',name='conv_6'))
model.add(BatchNormalization(name='batchnorm_6'))
model.add(Dense(units=36,activation='relu',name='relu_6'))
model.add(AveragePooling2D(pool_size=(2,1),strides=(4,1),name='avgpool_6'))
model.add(Flatten())

#Now adding in the fully-connected dense layer: This will act on the data extracted from the CNN in the prior stages
model.add(Dense(units= 30000,activation=None,use_bias=True))
model.summary()

#Last layer of the FCN is only showing the inputsx weight.  
#Investigating to see, if the bias is used. even though model summary is not displaying it.
#weights, biases = model.layers[25].get_weights()
#print(len(weights))
#print(len(biases))

#Build the model, we follow the paper, using Adam optimizer, for speeding up gradient descent
#And RMS as the cost function, metric to optimize against.

#Compile the model
model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

#Fit the model.
# Train the Model
#history = model.fit_generator(train_generator,validation_data = validation_generator, epochs = 10,steps_per_epoch = 1372, validation_steps = 350,verbose = 1 )
history = model.fit(XTrain_reshaped,YTrain_reshaped,epochs=15,verbose=1,validation_data=(XTest_reshaped,YTest_reshaped))

# Plot the chart for accuracy and loss on both training and validation
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()
                                    

Start defining the LSTM implementation:

RE-shape the Data so that it can be used in the LSTM:

The definition of the LSTM model follows below:

In [ ]:
model1 = Sequential(name="DNN using LSTM for Denoising");
#The LSTM->RELU-> return sequential ouput.
model1.add(LSTM(140,input_shape=(30000,1,),name="lstm_1_relu_1",activation='relu',return_sequences=True)) 

# sequential output/input -> LSTM2->Relu -> flattened output (return_sequence = false, by default)
#The output of LSTM2 is a flattened output -> Fully connected layer.
model1.add(LSTM(140,name="lstm_2_relu_2",activation='relu'))

#Add the fully connected layer of 30000 
model1.add(Dense(30000,activation=None))
model1.summary()

#Build the model, we follow the paper, using Adam optimizer, for speeding up gradient descent
#And RMS as the cost function, metric to optimize against.

#Compile the model
model1.compile(optimizer='adam',loss='mean_squared_error')

#Fit the model.
# Train the Model
history = model.fit_generator(train_generator,validation_data = validation_generator, epochs = 10,steps_per_epoch = 1372, validation_steps = 350,verbose = 1 )

# Plot the chart for accuracy and loss on both training and validation
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()


Helpful references:

1)This explains why last dense layer has no activation function:
  - we primarily do that if we are trying to accuractely do regression.

https://stats.stackexchange.com/questions/361066/what-is-the-point-of-having-a-dense-layer-in-a-neural-network-with-no-activation

2)This video shows how to implement a stacked LSTM model:
https://www.youtube.com/watch?v=BSpXCRTOLJA

3) This video shows the inner anatomy of an LSTM, primarily the sigmoid and Tanh, used for gating inside the LSTM:
https://www.youtube.com/watch?v=8HyCNIVRbSU&t=632s

4) The blogpost that references the video in 3. Is linked here:
https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21

5) This is the link from Andrew NG, I studied to understand, why we use BatchNormalization in the CNN:
https://www.youtube.com/watch?v=nUUqwaxLnWs


